<a href="https://colab.research.google.com/github/ddiestra/avila-dataset/blob/master/Ajuste_de_Hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ajuste de Hiperparametros

##Lectura de Datos

In [1]:
import urllib.request
import zipfile, urllib.request, shutil
from __future__ import absolute_import, division, print_function, unicode_literals


url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00577/codon_usage.csv.zip'
filename = 'codon_usage.csv.zip'

In [2]:
with urllib.request.urlopen(url) as response, open(filename, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)
    with zipfile.ZipFile(filename) as zf:
        zf.extractall()

In [3]:
!unzip codon_usage.csv

Archive:  codon_usage.csv
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
Archive:  codon_usage.csv.zip
replace codon_usage.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [13]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
labelencoder = LabelEncoder()

In [6]:
#Lectura del dataset
file_name = "codon_usage.csv"
dataset = pd.read_csv(file_name,  header=None)
# Eliminando columnas inválidas
columns = dataset.iloc[0]
dataset.columns = columns
dataset = dataset.drop([0],axis=0)

cols = dataset.columns[dataset.dtypes.eq(object)] # Seleccionamos las columnas con el tipo de datao : Object
dataset = dataset[cols].apply(pd.to_numeric, errors='ignore') #se transforma

# Se observa que hay datos str que no permiten manipular los demás como numéricos, por lo tanto se va a forzar.
dataset[['UUU', 'UUC']] = dataset[['UUU', 'UUC']].apply(pd.to_numeric, errors='coerce')
null_percentages = dataset['UUU'].isna().sum()
null_percentages

/Users/yape/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


2

In [14]:
def preprocess_dataset(dataset):
  preprocessed_dataset = dataset.copy()

  numeric_columns = list()
  preprocessed_dataset = preprocessed_dataset.drop("SpeciesID", axis=1)
  label_encoder = LabelEncoder()

  for column in preprocessed_dataset.columns:
    if not pd.api.types.is_numeric_dtype(preprocessed_dataset[column]):
      preprocessed_dataset[column] = label_encoder.fit_transform(preprocessed_dataset[column])
    else:
     
     numeric_columns.append(column)   
     #Reemplazamos los nulos con la mediana sólo de aquellas variables numéricas
     median_of_numeric_columns = preprocessed_dataset[numeric_columns].median()
     preprocessed_dataset[numeric_columns] = preprocessed_dataset[numeric_columns].fillna(median_of_numeric_columns)
      #print("Para la columna '{}', la codificación fue: {}".format(column, dict(enumerate(label_encoder.classes_))))  
  
  return preprocessed_dataset

In [15]:
preprocessed_dataset = preprocess_dataset(dataset)

In [16]:
labelencoder = LabelEncoder()
labelencoder.fit(preprocessed_dataset['Kingdom']);
Y1 = labelencoder.transform(preprocessed_dataset["Kingdom"]) #Leemos la data de prueba y usamos un encoder para las clases 

In [17]:
labelencoder = LabelEncoder()
labelencoder.fit(preprocessed_dataset['DNAtype']);
Y2 = labelencoder.transform(preprocessed_dataset["DNAtype"]) #Leemos la data de prueba y usamos un encoder para las clases

In [18]:
preprocessed_dataset.drop(["Kingdom","DNAtype"], axis=1, inplace=True)
X = preprocessed_dataset
X.head() #Separamos la data de la clase en variables X y Y1,Y2

,Ncodons,SpeciesName,UUU,UUC,UUA,UUG,CUU,CUC,CUA,CUG,...,CGG,AGA,AGG,GAU,GAC,GAA,GAG,UAA,UAG,UGA
1,1995,2827,0.01654,0.01203,0.00050,0.00351,0.01203,0.03208,0.00100,0.04010,...,0.00451,0.01303,0.03559,0.01003,0.04612,0.01203,0.04361,0.00251,0.00050,0.00000
2,1474,1087,0.02714,0.01357,0.00068,0.00678,0.00407,0.02849,0.00204,0.04410,...,0.00136,0.01696,0.03596,0.01221,0.04545,0.01560,0.04410,0.00271,0.00068,0.00000
3,4862,8252,0.01974,0.02180,0.01357,0.01543,0.00782,0.01111,0.01028,0.01193,...,0.00596,0.01974,0.02489,0.03126,0.02036,0.02242,0.02468,0.00391,0.00000,0.00144
4,1915,5480,0.01775,0.02245,0.01619,0.00992,0.01567,0.01358,0.00940,0.01723,...,0.00366,0.01410,0.01671,0.03760,0.01932,0.03029,0.03446,0.00261,0.00157,0.00000
5,22831,7577,0.02816,0.01371,0.00767,0.03679,0.01380,0.00548,0.00473,0.02076,...,0.00604,0.01494,0.01734,0.04148,0.02483,0.03359,0.03679,0.00000,0.00044,0.00131


In [23]:
#Mediante el método "train_test_split" usaremos el 20% de la data para probar el modelo. El parámetro "random state" nos sirve para
#poder replicar la misma separación
X, Xt, Y, Yt = train_test_split(X, Y1, test_size=0.2, random_state=70)

##Eliminas algunas caracteristicas 
Basandonos en el colab anterior decidimos usar solo 5 de las caracteristicas 

In [24]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif, mutual_info_classif

In [25]:
filtrado = SelectKBest(mutual_info_classif, k=5).fit(X, Y)
X_new = filtrado.transform(X)
X_new[:5]

array([[2.515e+03, 7.440e-03, 7.040e-03, 8.630e-03, 7.400e-04],
       [3.886e+03, 1.320e-02, 1.526e-02, 7.010e-03, 0.000e+00],
       [7.924e+03, 2.427e-02, 9.670e-03, 4.320e-03, 6.200e-04],
       [3.167e+03, 1.320e-02, 1.617e-02, 9.730e-03, 5.600e-04],
       [5.969e+03, 4.620e-03, 2.310e-03, 4.620e-03, 2.900e-04]])

In [26]:
Xt_new = filtrado.transform(Xt)
Xt_new[:5]

array([[6.9260e+03, 1.5800e-03, 1.1800e-03, 2.7600e-03, 2.7600e-03],
       [1.2735e+04, 6.8700e-02, 0.0000e+00, 7.4000e-04, 2.4510e-02],
       [9.2920e+03, 1.8720e-02, 2.1670e-02, 9.8500e-03, 1.9700e-03],
       [3.5270e+03, 1.0270e-02, 7.3700e-03, 1.5950e-02, 1.9000e-04],
       [9.4920e+03, 1.5060e-02, 2.2590e-02, 6.0200e-03, 0.0000e+00]])

##Entrenamos cada modelo

In [27]:
from sklearn.model_selection import GridSearchCV

###Decision Tree

Decission Tree Base

In [28]:
dt = DecisionTreeClassifier()
dt.fit(X_new, Y)
print('Max Depth: ', dt.tree_.max_depth)
print('Score: ',dt.score(Xt_new,Yt))
print('F1: ', f1_score(Yt, dt.predict(Xt_new), average='weighted'))

Max Depth:  29
Score:  0.6366078280890253
F1:  0.6361008181597602


Busqueda por Grilla

In [29]:
params = {"max_depth": [10,11,12,13,14,15,16,17, None], "max_features": [3,4,5], "min_samples_leaf": [3,4,5,6,7], "criterion": ["gini", "entropy"]}
tree = DecisionTreeClassifier()
dt_cv = GridSearchCV(tree, params, cv=5)
dt_cv.fit(X_new, Y)
print("Mejores Parametros: {}".format(dt_cv.best_params_))
print("Mejor Score {}".format(dt_cv.best_score_))

Mejores Parametros: {'criterion': 'gini', 'max_depth': 11, 'max_features': 5, 'min_samples_leaf': 7}
Mejor Score 0.6721347988787473


Reentrenamos como los parametros encontrados

In [37]:
dt_final = DecisionTreeClassifier(max_depth=11, max_features=5, min_samples_leaf=7, criterion='gini')
dt_final.fit(X_new, Y)
print('Score: ',dt_final.score(Xt_new,Yt))
print('F1: ', f1_score(Yt, dt_final.predict(Xt_new), average='weighted'))

Score:  0.6561780506523408
F1:  0.641835143164655


###Random Forest

Random Forest Base

In [38]:
rt = RandomForestClassifier(40, n_jobs=-1, oob_score=True)
rt.fit(X_new, Y)
print('Score: ',dt.score(Xt_new,Yt))
print('F1: ', f1_score(Yt, rt.predict(Xt_new), average='weighted'))

Score:  0.6366078280890253
F1:  0.7049453485097168


Busqueda por Grilla

In [39]:
params = {"n_estimators": [40,50,100,150,200], "max_depth": [10,11,12,13,14,15,16,17], "max_features": [3,4,5], "min_samples_leaf": [3,4,5,6,7]}
forest = RandomForestClassifier(oob_score=True)
rf_cv = GridSearchCV(forest, params, cv=5)
rf_cv.fit(X_new, Y)
print("Mejores Parametros: {}".format(rf_cv.best_params_))
print("Mejor Score {}".format(rf_cv.best_score_))

Mejores Parametros: {'max_depth': 16, 'max_features': 4, 'min_samples_leaf': 3, 'n_estimators': 150}
Mejor Score 0.7204004473964016


In [40]:
rf_final = RandomForestClassifier(oob_score=True, max_depth=16, max_features=4, min_samples_leaf=3, n_estimators=150)
rf_final.fit(X_new, Y)
print('Score: ',rf_final.score(Xt_new,Yt))
print('F1: ', f1_score(Yt, rf_final.predict(Xt_new), average='weighted'))

Score:  0.7106676899462778
F1:  0.7003540888993672


###KNN

KNN BASE

In [34]:
knn = KNeighborsClassifier(4)
knn.fit(X_new, Y)
print('Score: ',knn.score(Xt_new,Yt))
print('F1: ', f1_score(Yt, knn.predict(Xt_new), average='weighted'))

Score:  0.7302379125095932
F1:  0.7303226930282792


In [35]:
params = {"n_neighbors": [3,4,5, 6,8,10], "weights": ['uniform', 'distance'], "metric": ['euclidean', 'manhattan']}
forest = KNeighborsClassifier()
knn_cv = GridSearchCV(forest, params, cv=5)
knn_cv.fit(X_new, Y)
print("Mejores Parametros: {}".format(knn_cv.best_params_))
print("Mejor Score {}".format(knn_cv.best_score_))

Mejores Parametros: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
Mejor Score 0.752733905006513


In [36]:
knn_final = KNeighborsClassifier(n_neighbors=4, metric='euclidean',weights='distance')
knn_final.fit(X_new, Y)
print('Score: ',knn_final.score(Xt_new,Yt))
print('F1: ', f1_score(Yt, knn_final.predict(Xt_new), average='weighted'))

Score:  0.7647735993860323
F1:  0.7639740366990467
